# Mi cadena de markov

## Bibliotecas

In [93]:
# Warnings ---------------------------------------------------------------------------------------------------

import warnings
warnings.filterwarnings("ignore")

# Lectura y manipulación de datos ----------------------------------------------------------------------------

import pandas as pd

# Descomposición espectral -----------------------------------------------------------------------------------

import numpy as np
from numpy.linalg import eig, inv


## Lectura de la base de datos

In [94]:
data = pd.read_parquet('data/tec_estocasticos.parquet', engine='pyarrow')

In [3]:
data

,periodo,cliente_id,material_id,tipo_cliente
0,05-2022,4894.0,22.0,Distribuidor
1,05-2022,4769.0,17.0,Distribuidor
2,05-2022,4823.0,227.0,Distribuidor
3,08-2022,4816.0,340.0,Distribuidor
4,08-2022,4888.0,270.0,Distribuidor
...,...,...,...,...
6973895,05-2022,1816.0,1861.0,Farmacia
6973896,05-2022,1725.0,1373.0,Hospital
6973897,05-2022,588.0,2249.0,Hospital
6973898,05-2022,565.0,191.0,Hospital


In [99]:
data['periodo'] = pd.to_datetime(data['periodo'])
data.sort_values(by='periodo', inplace=True)
data.dropna(inplace=True)
data.reset_index(inplace=True)
data.drop('index', axis = 1,inplace=True)
data

,periodo,cliente_id,material_id,tipo_cliente
0,2021-01-01,800.0,317.0,Hospital
1,2021-01-01,782.0,2043.0,Hospital
2,2021-01-01,303.0,1352.0,Hospital
3,2021-01-01,314.0,1577.0,Hospital
4,2021-01-01,122.0,1504.0,Hospital
...,...,...,...,...
6955591,2023-09-01,4406.0,317.0,Hospital
6955592,2023-09-01,587.0,2321.0,Hospital
6955593,2023-09-01,587.0,1359.0,Hospital
6955594,2023-09-01,2421.0,341.0,Hospital


In [98]:
data

,periodo,cliente_id,material_id,tipo_cliente
0,05-2022,4894.0,22.0,Distribuidor
1,05-2022,4769.0,17.0,Distribuidor
2,05-2022,4823.0,227.0,Distribuidor
3,08-2022,4816.0,340.0,Distribuidor
4,08-2022,4888.0,270.0,Distribuidor
...,...,...,...,...
6973895,05-2022,1816.0,1861.0,Farmacia
6973896,05-2022,1725.0,1373.0,Hospital
6973897,05-2022,588.0,2249.0,Hospital
6973898,05-2022,565.0,191.0,Hospital


### Tipos de Cliente

In [6]:
data['tipo_cliente'].unique()

array(['Hospital', 'Farmacia', 'Distribuidor', 'Otro'], dtype=object)

## Función para obtener la matriz de transición de un cliente comprando un determinado producto

In [1]:
def matriz_transicion(tipo_cliente, cliente_id, material_id):
        
    # Subset

    cliente_tipo = data.loc[data['tipo_cliente'] == tipo_cliente]
    pruducto = cliente_tipo.loc[cliente_tipo['material_id'] == material_id]
    id_cliente = pruducto.loc[pruducto['cliente_id'] == cliente_id]
    id_cliente.reset_index(inplace = True)
    id_cliente.drop('index', axis = 1, inplace = True)
    
    # Fecha de inicio
    fecha_inicio = pd.Timestamp(2021, 1, 1)
    # Fecha de finalización
    fecha_fin = pd.Timestamp(2023, 9, 1)

    frecuencia = pd.DateOffset(months=1)

    fechas = []

    while fecha_inicio <= fecha_fin:
        fechas.append(fecha_inicio)
        fecha_inicio += frecuencia

    fechas

    periodos = {}

    for k,v in enumerate(fechas):
        periodos[v] = k 

    t = [1 for x in range(len(fechas))]

    indices = [periodos[x] for x in id_cliente['periodo']]

    for i in indices:
        t[i] = 0
    
    estados = pd.DataFrame()

    estados['t'] = t
    estados['t_1'] = t
    
    Xt = estados['t'][0:-1].reset_index(drop=True).rename('X_t')
    Xt_1 = estados['t_1'][1::].reset_index(drop=True).rename('X_t+1')
    
    new_data=pd.concat((Xt, Xt_1), axis=1)
    
    matriz_transicion = new_data.groupby('X_t').value_counts(normalize=True).unstack(level='X_t+1')
    matriz_transicion= matriz_transicion.fillna(0)
    
    return matriz_transicion

In [2]:
matriz_transicion('Hospital', 800.0 ,317.0)

NameError: name 'data' is not defined

In [148]:
matriz_transicion('Farmacia', 2000.0 ,259.0)

,X_t,X_t+1
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [117]:
t

NameError: name 't' is not defined

In [129]:
tipo_cliente = 'Farmacia'
cliente_id = 2000.0
material_id = 259.0

cliente_tipo = data.loc[data['tipo_cliente'] == tipo_cliente]
pruducto = cliente_tipo.loc[cliente_tipo['material_id'] == material_id]
id_cliente = pruducto.loc[pruducto['cliente_id'] == cliente_id]
id_cliente.reset_index(inplace = True)
    
id_cliente.drop('index', axis = 1, inplace = True)

id_cliente

,periodo,cliente_id,material_id,tipo_cliente
0,2021-01-01,2000.0,259.0,Farmacia
1,2021-02-01,2000.0,259.0,Farmacia
2,2021-03-01,2000.0,259.0,Farmacia
3,2021-04-01,2000.0,259.0,Farmacia
4,2021-05-01,2000.0,259.0,Farmacia
5,2021-06-01,2000.0,259.0,Farmacia
6,2021-07-01,2000.0,259.0,Farmacia
7,2021-08-01,2000.0,259.0,Farmacia
8,2021-09-01,2000.0,259.0,Farmacia
9,2021-10-01,2000.0,259.0,Farmacia


In [65]:
t = []

for x in range(0, len(id_cliente['periodo'])-1):
    if (id_cliente['periodo'][x+1] - id_cliente['periodo'][x]).days <= 31:
        t.append(0)
    else:
        for _ in range(((id_cliente['periodo'][x+1] - id_cliente['periodo'][x]).days // 30)):
            t.append(1)
        t.append(0)
        
t

6

In [26]:
data

,periodo,cliente_id,material_id,tipo_cliente
0,2021-01-01,800.0,317.0,Hospital
1,2021-01-01,782.0,2043.0,Hospital
2,2021-01-01,303.0,1352.0,Hospital
3,2021-01-01,314.0,1577.0,Hospital
4,2021-01-01,122.0,1504.0,Hospital
...,...,...,...,...
6955591,2023-09-01,4406.0,317.0,Hospital
6955592,2023-09-01,587.0,2321.0,Hospital
6955593,2023-09-01,587.0,1359.0,Hospital
6955594,2023-09-01,2421.0,341.0,Hospital


In [102]:
start = pd.DataFrame()

start['periodo'] = [pd.Timestamp(2021, 1, 1)]
start['cliente_id'] = [cliente_id]
start['material_id'] = [material_id]
start['tipo_cliente'] = [tipo_cliente]

end = pd.DataFrame()
end['periodo'] = [pd.Timestamp(2023, 9, 1)]
end['cliente_id'] = [cliente_id]
end['material_id'] = [material_id]
end['tipo_cliente'] = [tipo_cliente]


id_cliente = pd.concat([start, id_cliente, end], axis = 0)
id_cliente.reset_index(inplace = True)

In [103]:
id_cliente

,index,periodo,cliente_id,material_id,tipo_cliente
0,0,2021-01-01,2000.0,259.0,Farmacia
1,0,2021-01-01,2000.0,259.0,Farmacia
2,1,2021-02-01,2000.0,259.0,Farmacia
3,2,2021-03-01,2000.0,259.0,Farmacia
4,3,2021-04-01,2000.0,259.0,Farmacia
5,4,2021-05-01,2000.0,259.0,Farmacia
6,5,2021-06-01,2000.0,259.0,Farmacia
7,6,2021-07-01,2000.0,259.0,Farmacia
8,7,2021-08-01,2000.0,259.0,Farmacia
9,8,2021-09-01,2000.0,259.0,Farmacia


In [131]:
import pandas as pd

# Fecha de inicio
fecha_inicio = pd.Timestamp(2021, 1, 1)

# Fecha de finalización
fecha_fin = pd.Timestamp(2023, 9, 1)

# Frecuencia mensual
frecuencia = pd.DateOffset(months=1)

# Lista para almacenar las fechas
fechas = []

# Bucle para generar las fechas mensuales
while fecha_inicio <= fecha_fin:
    fechas.append(fecha_inicio)
    fecha_inicio += frecuencia

fechas

[Timestamp('2021-01-01 00:00:00'),
 Timestamp('2021-02-01 00:00:00'),
 Timestamp('2021-03-01 00:00:00'),
 Timestamp('2021-04-01 00:00:00'),
 Timestamp('2021-05-01 00:00:00'),
 Timestamp('2021-06-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-08-01 00:00:00'),
 Timestamp('2021-09-01 00:00:00'),
 Timestamp('2021-10-01 00:00:00'),
 Timestamp('2021-11-01 00:00:00'),
 Timestamp('2021-12-01 00:00:00'),
 Timestamp('2022-01-01 00:00:00'),
 Timestamp('2022-02-01 00:00:00'),
 Timestamp('2022-03-01 00:00:00'),
 Timestamp('2022-04-01 00:00:00'),
 Timestamp('2022-05-01 00:00:00'),
 Timestamp('2022-06-01 00:00:00'),
 Timestamp('2022-07-01 00:00:00'),
 Timestamp('2022-08-01 00:00:00'),
 Timestamp('2022-09-01 00:00:00'),
 Timestamp('2022-10-01 00:00:00'),
 Timestamp('2022-11-01 00:00:00'),
 Timestamp('2022-12-01 00:00:00'),
 Timestamp('2023-01-01 00:00:00'),
 Timestamp('2023-02-01 00:00:00'),
 Timestamp('2023-03-01 00:00:00'),
 Timestamp('2023-04-01 00:00:00'),
 Timestamp('2023-05-

In [133]:
periodos = {}

for k,v in enumerate(fechas):
    periodos[v] = k 
    
periodos

{Timestamp('2021-01-01 00:00:00'): 0,
 Timestamp('2021-02-01 00:00:00'): 1,
 Timestamp('2021-03-01 00:00:00'): 2,
 Timestamp('2021-04-01 00:00:00'): 3,
 Timestamp('2021-05-01 00:00:00'): 4,
 Timestamp('2021-06-01 00:00:00'): 5,
 Timestamp('2021-07-01 00:00:00'): 6,
 Timestamp('2021-08-01 00:00:00'): 7,
 Timestamp('2021-09-01 00:00:00'): 8,
 Timestamp('2021-10-01 00:00:00'): 9,
 Timestamp('2021-11-01 00:00:00'): 10,
 Timestamp('2021-12-01 00:00:00'): 11,
 Timestamp('2022-01-01 00:00:00'): 12,
 Timestamp('2022-02-01 00:00:00'): 13,
 Timestamp('2022-03-01 00:00:00'): 14,
 Timestamp('2022-04-01 00:00:00'): 15,
 Timestamp('2022-05-01 00:00:00'): 16,
 Timestamp('2022-06-01 00:00:00'): 17,
 Timestamp('2022-07-01 00:00:00'): 18,
 Timestamp('2022-08-01 00:00:00'): 19,
 Timestamp('2022-09-01 00:00:00'): 20,
 Timestamp('2022-10-01 00:00:00'): 21,
 Timestamp('2022-11-01 00:00:00'): 22,
 Timestamp('2022-12-01 00:00:00'): 23,
 Timestamp('2023-01-01 00:00:00'): 24,
 Timestamp('2023-02-01 00:00:00'): 

In [134]:
id_cliente

,periodo,cliente_id,material_id,tipo_cliente
0,2021-01-01,2000.0,259.0,Farmacia
1,2021-02-01,2000.0,259.0,Farmacia
2,2021-03-01,2000.0,259.0,Farmacia
3,2021-04-01,2000.0,259.0,Farmacia
4,2021-05-01,2000.0,259.0,Farmacia
5,2021-06-01,2000.0,259.0,Farmacia
6,2021-07-01,2000.0,259.0,Farmacia
7,2021-08-01,2000.0,259.0,Farmacia
8,2021-09-01,2000.0,259.0,Farmacia
9,2021-10-01,2000.0,259.0,Farmacia


In [139]:
periodos[id_cliente[0]]

KeyError: 0

In [142]:
# Fecha de inicio
fecha_inicio = pd.Timestamp(2021, 1, 1)

# Fecha de finalización
fecha_fin = pd.Timestamp(2023, 9, 1)

frecuencia = pd.DateOffset(months=1)

fechas = []

while fecha_inicio <= fecha_fin:
    fechas.append(fecha_inicio)
    fecha_inicio += frecuencia

fechas

periodos = {}

for k,v in enumerate(fechas):
    periodos[v] = k 
    
t = [1 for x in range(len(fechas))]

indices = [periodos[x] for x in id_cliente['periodo']]

for i in indices:
    t[i] = 0
    
t

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [80]:
(id_cliente['periodo'][13] - id_cliente['periodo'][12]).days // 30

21

In [123]:
t = []

for x in range(0, len(id_cliente['periodo'])-1):
    if (id_cliente['periodo'][x+1] - id_cliente['periodo'][x]).days <= 31:
        t.append(0)
    else:
        for _ in range(((id_cliente['periodo'][x+1] - id_cliente['periodo'][x]).days // 30)-1):
            t.append(1)
        #t.append(0)
        
t

[0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0]

In [124]:
t_1 = []

for x in range(0, len(id_cliente['periodo'])-1):
    if (id_cliente['periodo'][x+1] - id_cliente['periodo'][x]).days <= 31:
        t_1.append(0)
    else:
        for _ in range(((id_cliente['periodo'][x+1] - id_cliente['periodo'][x]).days // 30)-1):
            t_1.append(1)
        #t_1.append(1)
        
t_1

[0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0]

In [125]:
len(t_1)

30

In [126]:
len(t)

30

In [128]:
estados = pd.DataFrame()

estados['t'] = t
estados['t_1'] = t_1

Xt = estados['t'][0:-1].reset_index(drop=True).rename('X_t')
Xt_1 = estados['t_1'][1::].reset_index(drop=True).rename('X_t+1')

new_data=pd.concat((Xt, Xt_1), axis=1)

new_data

,X_t,X_t+1
0,0,1
1,1,1
2,1,1
3,1,1
4,1,1
5,1,1
6,1,0
7,0,0
8,0,1
9,1,0


## Dropdown

In [7]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

In [8]:
# Markdown Funcional

# Opciones

tipo_cliente = list(data['tipo_cliente'].unique())

cliente_id = list(data['cliente_id'].unique()) 

material_id = list(data['material_id'].unique()) 

pasos_t = list(range(1,51))

# Botones

dropdown_tipo_cliente = widgets.Dropdown(
    options=tipo_cliente,
    description="tipo_cliente:"
)

dropdown_cliente_id = widgets.Dropdown(
    options=cliente_id,
    description="cliente_id:"
)

dropdown_material_id = widgets.Dropdown(
    options= material_id,
    description="material_id:"
)

dropdown_pasos = widgets.Dropdown(
    options=pasos_t,
    description="pasos:"
)


# Función para la visualización con parámetros
def valores(tipo_cliente, cliente_id, material_id, pasos_t):
    
    try: 
        Lambda, Q = eig(matriz_transicion(tipo_cliente, cliente_id, material_id))
        Q_1 = inv(Q)
        Lambda = np.diag(Lambda)
        PP = np.matmul(np.matmul(Q, Lambda), Q_1)

        Lambda_n = Lambda**pasos_t
        P_n = np.matmul(np.matmul(Q, Lambda_n), Q_1)

        df = pd.DataFrame(P_n.round(decimals = 4), index=['Compra', 'No compra'])
        df.rename(columns = {0:'Compra'}, inplace = True)
        df.rename(columns = {1:'No Compra'}, inplace = True)
    
    except:
        return print('No existe información de ese cliente comprando ese producto')
    return print(df)

widgets.interactive(valores, 
                    tipo_cliente=dropdown_tipo_cliente, 
                    cliente_id=dropdown_cliente_id,
                    material_id=dropdown_material_id,
                    pasos_t = dropdown_pasos)

interactive(children=(Dropdown(description='tipo_cliente:', options=('Hospital', 'Farmacia', 'Distribuidor', '…